# Import Dependencies

In [21]:
import warnings
warnings.filterwarnings('ignore')
import keras
import matplotlib.pyplot as plt

## Define Types

In [117]:
from typing import Tuple
ImageShape = Tuple[int, int]
GrayScaleImageShape = Tuple[int, int, int]

# MNIST Sandbox Baseline Example

In [13]:
from keras.datasets import mnist
import matplotlib.pyplot as plt

In [116]:
from typing import Tuple
import numpy as np
Dataset = Tuple[np.ndarray, np.ndarray]

#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(f"The shape of X_train is {X_train.shape}")
print(f"The shape of y_train is {y_train.shape}")
print(f"The shape of X_test is {X_test.shape}")
print(f"The shape of y_test is {y_test.shape} - some example targets: {y_test[:5]}")
mnist_image_shape: ImageShape = X_train.shape[1:]
print(mnist_image_shape)

The shape of X_train is (60000, 28, 28)
The shape of y_train is (60000,)
The shape of X_test is (10000, 28, 28)
The shape of y_test is (10000,) - some example targets: [7 2 1 0 4]
(28, 28)


In [60]:
from keras.utils import to_categorical

OneHotEncodedTarget = np.ndarray
Categories = int
encoded_y_train: OneHotEncodedTarget = to_categorical(y_train)
encoded_y_test: OneHotEncodedTarget = to_categorical(y_test)
print(f"One-hot encoding y_train {y_train.shape} -> {encoded_y_train.shape}")
print(f"One-hot encoding y_test {y_test.shape} -> {encoded_y_test.shape}")

K: Categories = encoded_y_test.shape[1]

One-hot encoding y_train (60000,) -> (60000, 10)
One-hot encoding y_test (10000,) -> (10000, 10)


# Vanilla CNN Implementation

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Input
from tensorflow.python.framework.ops import Tensor
import warnings
warnings.filterwarnings('ignore')

# define model architecture and hyperparameters
NUM_FILTERS_L1 = 64
NUM_FILTERS_L2 = 32
KERNEL_SIZE = 3

# the images are 28 x 28 (pixel size) x 1 (grayscale - if RGB, then 3)
input_dims: GrayScaleImageShape = (28,28,1)

def build_vanilla_cnn(filters_layer1:int, filters_layer2:int, kernel_size:int, input_dims: GrayScaleImageShape)-> Model:
    inputs: Tensor = Input(shape=input_dims)
    x: Tensor = Dense(128, activation='relu')(inputs)
    x: Tensor = Conv2D(filters=filters_layer1, kernel_size=kernel_size, activation='relu')(x)
    x: Tensor = Conv2D(filters=filters_layer2, kernel_size=kernel_size, activation='relu')(x)
    x: Tensor = Flatten()(x)
    predictions = Dense(K, activation="softmax")(x)
    print(predictions)

    #compile model using accuracy to measure model performance
    model: Model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
    return model

model: Model = build_vanilla_cnn(NUM_FILTERS_L1, NUM_FILTERS_L2, KERNEL_SIZE, input_dims)

# Example Attention Model

In [127]:
input_dims

(28, 28, 1)

In [134]:
from keras.layers import merge

def build_model(input_dim):
    inputs = Input(shape=input_dim)

    # ATTENTION PART STARTS HERE
    attention_probs = Dense(input_dim, activation='softmax', name='attention_vec')(inputs)
    attention_mul = merge([inputs, attention_probs], output_shape=32, name='attention_mul', mode='mul')
    # ATTENTION PART FINISHES HERE

    attention_mul = Dense(64)(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model

inputs = Input(shape=input_dims)
attention_probs = Dense(input_dims, activation='softmax', name='attention_vec')(inputs)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

## Compile and Fit Model

In [121]:
X_train.reshape((60000,1,28,28))

def expand_tensor_shape(X_train: np.ndarray)-> np.ndarray:
    new_shape: Tuple = X_train.shape + (1,)
    print(f"Expanding shape from {X_train.shape} to {new_shape}")
    return X_train.reshape(new_shape)

X_train_expanded: np.ndarray = expand_tensor_shape(X_train)
X_test_expanded: np.ndarray = expand_tensor_shape(X_test)

Expanding shape from (60000, 28, 28) to (60000, 28, 28, 1)
Expanding shape from (10000, 28, 28) to (10000, 28, 28, 1)


In [122]:

# train model and retrieve history
from keras.callbacks import History
history: History = model.fit(X_train_expanded, encoded_y_train, 
                             validation_data=(X_test_expanded, encoded_y_test), epochs=2, batch_size=2058)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
32928/60000 [===============>..............] - ETA: 1:37 - loss: 5.4845 - acc: 0.4824

KeyboardInterrupt: 

{'val_loss': [0.11421830420212928,
  0.1322451029705997,
  0.15506393317956932,
  0.19153590463257378,
  0.20397465853056024],
 'val_acc': [0.9787, 0.9788, 0.9785, 0.9771, 0.9742],
 'loss': [0.027731930499821027,
  0.018710533776183872,
  0.018969004292929897,
  0.018248560158168024,
  0.023170674585329343],
 'acc': [0.9920166666666667,
  0.9945,
  0.9945333333333334,
  0.9953666666666666,
  0.9950166666666667]}

# FEI Face Dataset

In [12]:
from PIL.JpegImagePlugin import JpegImageFile

image: JpegImageFile = load_img('1-01.jpg')

PIL.JpegImagePlugin.JpegImageFile